In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1203945/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_48_0,0.999817,0.813337,0.821625,0.998442,0.000792,1.248219,0.638038,0.003596,0.031985,0.028136,1.011702,0.028579,144.282779,285.617952,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_36_0,0.999808,0.806701,0.807898,0.999850,0.000830,1.292589,0.687138,0.001063,0.037221,0.028804,1.012263,0.029257,144.189032,285.524205,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_41_9,0.999803,0.812900,0.828704,0.999467,0.000853,1.251139,0.612717,0.002643,0.036186,0.029207,1.012609,0.029667,144.133448,285.468621,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_35_9,0.999801,0.806600,0.808244,0.999724,0.000860,1.293264,0.685900,0.000978,0.040129,0.029324,1.012711,0.029786,144.117372,285.452545,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_41_8,0.999791,0.813045,0.829225,0.999462,0.000905,1.250167,0.610852,0.002667,0.036887,0.030080,1.013374,0.030554,144.015595,285.350768,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,model_1_2_4,0.935449,0.738866,0.720027,0.912566,0.279488,1.746204,1.001447,0.295396,0.674757,0.528666,5.131237,0.536992,132.549593,273.884765,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
471,model_1_2_2,0.925917,0.721651,0.724403,0.921918,0.320760,1.861324,0.985794,0.263797,0.551710,0.566356,5.741293,0.575276,132.274127,273.609299,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
472,model_1_2_3,0.925456,0.721992,0.721497,0.916564,0.322757,1.859038,0.996188,0.281888,0.580614,0.568117,5.770822,0.577065,132.261710,273.596882,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
473,model_1_2_1,0.901057,0.699583,0.726291,0.869579,0.428396,2.008888,0.979041,0.440625,0.628397,0.654520,7.332323,0.664828,131.695413,273.030585,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
